In [1]:
import os, sys
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

current_directory = os.getcwd()
root_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
sys.path.append(root_directory)

c:\Users\mohammed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import os
import pandas as pd

# Directory containing the CSV files
directory = r'c:\Users\mohammed\OneDrive\Documents\QFM -S2\BankReviewIntelligence\ScrapperService\production_standalone\row_data\parcket\2024-06-08'

# List to hold the data from each CSV file
data_frames = []

# Iterate over all files in the directory
for country in os.listdir(directory):
    for city in os.listdir(directory+"\\"+country):
        if city.endswith(".parquet"):
            file_path = os.path.join(directory+"\\"+country, city)
            df = pd.read_parquet(file_path)
            data_frames.append(df)

# Concatenate all DataFrames
all_data = pd.concat(data_frames, ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
all_data.to_parquet('concatenated_data.parquet', index=False)
print(all_data.shape)
#all_data.head()

(1710, 14)


* Preprocessing

In [5]:
from parquet_preprocessing import preprocess_dataframe
all_data = pd.read_parquet('concatenated_data.parquet')
df = preprocess_dataframe(all_data)

No match for: il y a 5 ans
No match for: il y a 5 ans
No match for: il y a 2 ans
No match for: il y a 3 ans
No match for: il y a 2 ans
No match for: il y a 5 ans
No match for: il y a 6 ans
No match for: il y a 5 ans
No match for: il y a 5 ans
No match for: il y a 2 ans
No match for: il y a 6 ans
No match for: il y a 7 ans
No match for: il y a 7 ans
No match for: il y a 6 ans
No match for: il y a 9 ans
No match for: il y a 5 ans
No match for: il y a 6 ans
No match for: il y a 5 ans
No match for: il y a 3 ans
No match for: il y a 9 mois
No match for: il y a 6 ans
No match for: il y a 6 ans
No match for: il y a 10 mois
No match for: il y a 9 ans
No match for: il y a 6 ans
No match for: il y a 4 ans
No match for: il y a 4 ans
No match for: il y a 9 ans
No match for: il y a 8 ans
No match for: il y a 9 ans
No match for: il y a 6 ans
No match for: il y a 6 ans
No match for: il y a 6 ans
No match for: il y a 7 ans
No match for: il y a 6 ans
No match for: il y a 2 ans
No match for: il y a 11 a

In [3]:
df["Reviewer_Publish_Date"].unique()
#all_data["Reviewer_Publish_Date"].unique()

array([None], dtype=object)

In [18]:
import re
from datetime import datetime, timedelta

def parse_relative_date(relative_date_str):
    print(f"Processing: -{relative_date_str}-")
    if not relative_date_str or not isinstance(relative_date_str, str):
        return "0000-00-002"
    
    match = re.search(r"il y a (\w+|\d+) (\w+)", relative_date_str)
    if not match:
        print(f"No match for: {relative_date_str}")
        return "0000-00-001"
    
    match1 = 1 if match.group(1) in ["un", "une"]  else match.group(1)
    quantity = int(match1)
    unit = match.group(2).lower()
    now = datetime.now()
    
    if unit in ['ans', 'an']:
        return (now - timedelta(days=quantity * 365)).strftime('%Y-%m-%d')
    elif unit == 'mois':
        return (now - timedelta(days=quantity * 30)).strftime('%Y-%m-%d')
    elif unit in ['semaines', 'semaine']:
        return (now - timedelta(weeks=quantity)).strftime('%Y-%m-%d')
    elif unit in ['jours', 'jour']:
        return (now - timedelta(days=quantity)).strftime('%Y-%m-%d')
    elif unit in ['heures', 'heure']:
        return (now - timedelta(hours=quantity)).strftime('%Y-%m-%d')
    elif unit in ['minutes', 'minute']:
        return (now - timedelta(minutes=quantity)).strftime('%Y-%m-%d')
    else:
        print(f"Unrecognized unit: {unit}")
        return "0000-00-00"

In [20]:
parse_relative_date("il y a 20 ans")

Processing: -il y a 20 ans-


'2004-07-02'

In [14]:
str(x).split(" ")[0]

'2024-06-25'

In [10]:
all_data["Reviewer_Publish_Date"].apply(lambda x: parse_relative_date(x)).unique()

il y a un an
il y a 5 ans
il y a 5 ans
il y a 2 ans
il y a 3 ans
il y a 2 ans
il y a 5 ans
il y a 6 ans
il y a 5 ans
il y a 5 ans
il y a un an
il y a 2 ans
il y a 6 ans
il y a 7 ans
il y a 7 ans
il y a 6 ans
il y a 9 ans
il y a 5 ans
il y a 6 ans
il y a 5 ans
il y a 3 ans
il y a 9 mois
il y a 6 ans
il y a 6 ans
il y a 10 mois
il y a 9 ans
il y a un an
il y a 6 ans
il y a 4 ans
il y a 4 ans
il y a 9 ans
il y a 8 ans
il y a 9 ans
il y a 6 ans
il y a 6 ans
il y a 6 ans
il y a 7 ans
il y a 6 ans
il y a 2 ans
il y a un an
il y a 11 ans
il y a 7 ans
il y a 2 ans
il y a 2 ans
il y a un an
il y a 5 ans
il y a 5 ans
il y a 9 ans
il y a 10 mois
il y a 3 ans
il y a 6 ans
il y a un an
il y a 6 ans
il y a 6 ans
il y a 7 ans
il y a 7 ans
il y a 7 ans
il y a 9 ans
il y a 11 mois
il y a 6 ans
il y a 8 ans
il y a 6 ans
il y a 6 ans
il y a 4 ans
il y a 5 ans
il y a 8 ans
il y a 6 ans
il y a 9 ans
il y a 7 ans
il y a 5 ans
il y a 6 ans
il y a 3 ans
il y a 2 ans
il y a 6 ans
il y a 9 ans
il y a 6 ans
il y

array(['0000-00-00'], dtype=object)

In [6]:
import re
from datetime import datetime, timedelta

def parse_relative_date(relative_date_str):
    print(f"Processing: -{relative_date_str}-")
    if not relative_date_str or not isinstance(relative_date_str, str):
        return "0000-00-00"
    
    # Refined regex pattern to capture both words and numbers
    match = re.search(r"il y a (\d+|un|une) (\w+)", relative_date_str)
    if not match:
        print(f"No match for: {relative_date_str}")
        return "0000-00-00"
    
    # Convert 'un' or 'une' to 1
    quantity_str = match.group(1)
    if quantity_str in ["un", "une"]:
        quantity = 1
    else:
        quantity = int(quantity_str)
    
    unit = match.group(2).lower()
    now = datetime.now()
    
    if unit in ['ans', 'an']:
        return (now - timedelta(days=quantity * 365)).strftime('%Y-%m-%d')
    elif unit == 'mois':
        return (now - timedelta(days=quantity * 30)).strftime('%Y-%m-%d')
    elif unit in ['semaines', 'semaine']:
        return (now - timedelta(weeks=quantity)).strftime('%Y-%m-%d')
    elif unit in ['jours', 'jour']:
        return (now - timedelta(days=quantity)).strftime('%Y-%m-%d')
    elif unit in ['heures', 'heure']:
        return (now - timedelta(hours=quantity)).strftime('%Y-%m-%d')
    elif unit in ['minutes', 'minute']:
        return (now - timedelta(minutes=quantity)).strftime('%Y-%m-%d')
    else:
        print(f"Unrecognized unit: {unit}")
        return "0000-00-00"

# Apply the function to the DataFrame
unique_dates = all_data["Reviewer_Publish_Date"].apply(lambda x: parse_relative_date(x)).unique()

print(unique_dates)


Processing: -il y a un an-
Processing: -il y a 5 ans-
No match for: il y a 5 ans
Processing: -il y a 5 ans-
No match for: il y a 5 ans
Processing: -il y a 2 ans-
No match for: il y a 2 ans
Processing: -il y a 3 ans-
No match for: il y a 3 ans
Processing: -il y a 2 ans-
No match for: il y a 2 ans
Processing: -il y a 5 ans-
No match for: il y a 5 ans
Processing: -il y a 6 ans-
No match for: il y a 6 ans
Processing: -il y a 5 ans-
No match for: il y a 5 ans
Processing: -il y a 5 ans-
No match for: il y a 5 ans
Processing: -il y a un an-
Processing: -il y a 2 ans-
No match for: il y a 2 ans
Processing: -il y a 6 ans-
No match for: il y a 6 ans
Processing: -il y a 7 ans-
No match for: il y a 7 ans
Processing: -il y a 7 ans-
No match for: il y a 7 ans
Processing: -il y a 6 ans-
No match for: il y a 6 ans
Processing: -il y a 9 ans-
No match for: il y a 9 ans
Processing: -il y a 5 ans-
No match for: il y a 5 ans
Processing: -il y a 6 ans-
No match for: il y a 6 ans
Processing: -il y a 5 ans-
N

In [18]:
match = re.search(r"il y a (\w+) (\w+)", 'il y a 2 ans')
if not match:
    print("0000-00-001")

In [3]:
import re
from datetime import datetime, timedelta

def parse_relative_date(relative_date_str):
    print(f"Processing: -{relative_date_str}-")
    if not relative_date_str or not isinstance(relative_date_str, str):
        return "0000-00-00"
    
    # Refined regex pattern to capture both words and numbers
    match = re.search(r"il y a (\d+|un|une) (\w+)", relative_date_str)
    if not match:
        print(f"No match for: {relative_date_str}")
        return "0000-00-00"
    
    # Convert 'un' or 'une' to 1
    quantity_str = match.group(1)
    if quantity_str in ["un", "une"]:
        quantity = 1
    else:
        quantity = int(quantity_str)
    
    unit = match.group(2).lower()
    now = datetime.now()
    
    if unit in ['ans', 'an']:
        return (now - timedelta(days=quantity * 365)).strftime('%Y-%m-%d')
    elif unit == 'mois':
        return (now - timedelta(days=quantity * 30)).strftime('%Y-%m-%d')
    elif unit in ['semaines', 'semaine']:
        return (now - timedelta(weeks=quantity)).strftime('%Y-%m-%d')
    elif unit in ['jours', 'jour']:
        return (now - timedelta(days=quantity)).strftime('%Y-%m-%d')
    elif unit in ['heures', 'heure']:
        return (now - timedelta(hours=quantity)).strftime('%Y-%m-%d')
    elif unit in ['minutes', 'minute']:
        return (now - timedelta(minutes=quantity)).strftime('%Y-%m-%d')
    else:
        print(f"Unrecognized unit: {unit}")
        return "0000-00-00"

# Example DataFrame
import pandas as pd
data = {
    'Reviewer_Publish_Date': [
        'il y a un an', 'il y a 2 ans', 'il y a 5 ans',
        'il y a une semaine', 'il y a 3 jours', 'il y a 2 heures'
    ]
}
all_data = pd.DataFrame(data)

# Apply the function to the DataFrame
unique_dates = all_data["Reviewer_Publish_Date"].apply(lambda x: parse_relative_date(x)).unique()

print(unique_dates)

Processing: -il y a un an-
Processing: -il y a 2 ans-
Processing: -il y a 5 ans-
Processing: -il y a une semaine-
Processing: -il y a 3 jours-
Processing: -il y a 2 heures-
['2023-06-28' '2022-06-28' '2019-06-29' '2024-06-20' '2024-06-24'
 '2024-06-27']
